In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6736,2021-01-09T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1256,205,1461,38937,...,2695,NaN,NaN,106322,1294528,858606.0,NaN,9.0,NaN,NaN
6737,2021-01-09T17:00:00,ITA,9,Toscana,43.769231,11.255889,792,139,931,8174,...,3840,NaN,NaN,124479,1956024,1087156.0,NaN,6.0,Positivi diagnosticati solo con test antigenic...,NaN
6738,2021-01-09T17:00:00,ITA,10,Umbria,43.106758,12.388247,279,47,326,3911,...,652,NaN,NaN,30649,526153,259676.0,NaN,6.0,NaN,NaN
6739,2021-01-09T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,50,1,51,386,...,388,NaN,NaN,7486,64752,39706.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6736,2021-01-09T17:00:00,19,Sicilia,1256,205,1461,38937,40398,726,1839,63229,2695,NaN,NaN,106322,1294528,858606.0,9.0,NaN,NaN
6737,2021-01-09T17:00:00,9,Toscana,792,139,931,8174,9105,-130,529,111534,3840,NaN,NaN,124479,1956024,1087156.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
6738,2021-01-09T17:00:00,10,Umbria,279,47,326,3911,4237,129,295,25760,652,NaN,NaN,30649,526153,259676.0,6.0,NaN,NaN
6739,2021-01-09T17:00:00,2,Valle d'Aosta,50,1,51,386,437,12,22,6661,388,NaN,NaN,7486,64752,39706.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-09', '2021-01-08')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-09T17:00:00,1,Piemonte,2689,172,2861,12968,15829,-1716,1532,182165,8165,NaN,NaN,206159,1734982,1006938.0,14.0,NaN,NaN
1,2021-01-09T17:00:00,2,Valle d'Aosta,50,1,51,386,437,12,22,6661,388,NaN,NaN,7486,64752,39706.0,0.0,NaN,NaN
2,2021-01-09T17:00:00,3,Lombardia,3577,456,4033,51404,55437,760,2506,416326,25728,NaN,NaN,497491,5023007,2664865.0,15.0,NaN,NaN
3,2021-01-09T17:00:00,5,Veneto,2540,349,2889,86562,89451,-1499,3100,187023,7345,NaN,NaN,283819,3451512,1255475.0,32.0,NaN,NaN
4,2021-01-09T17:00:00,6,Friuli Venezia Giulia,690,62,752,12138,12890,314,1015,40825,1876,NaN,NaN,55591,981950,353683.0,3.0,NaN,NaN
5,2021-01-09T17:00:00,7,Liguria,702,65,767,4377,5144,-61,526,55257,2999,NaN,NaN,63400,739849,332011.0,7.0,I 180827 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-09T17:00:00,8,Emilia-Romagna,2679,240,2919,55449,58368,1520,1790,122054,8279,NaN,NaN,188701,2685079,1264605.0,15.0,NaN,NaN
7,2021-01-09T17:00:00,9,Toscana,792,139,931,8174,9105,-130,529,111534,3840,NaN,NaN,124479,1956024,1087156.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-09T17:00:00,10,Umbria,279,47,326,3911,4237,129,295,25760,652,NaN,NaN,30649,526153,259676.0,6.0,NaN,NaN
9,2021-01-09T17:00:00,11,Marche,529,71,600,14006,14606,594,688,29837,1685,NaN,NaN,46128,571099,341285.0,5.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-08T17:00:00,1,Piemonte,2757,176,2933,14612,17545,-154,845,178949,8133,NaN,NaN,204627,1723609,1003245.0,5.0,NaN,NaN
1,2021-01-08T17:00:00,2,Valle d'Aosta,54,1,55,370,425,-9,25,6651,388,NaN,NaN,7464,64477,39592.0,0.0,NaN,NaN
2,2021-01-08T17:00:00,3,Lombardia,3436,466,3902,50775,54677,435,1963,414643,25665,NaN,NaN,494985,4998160,2656804.0,25.0,NaN,NaN
3,2021-01-08T17:00:00,5,Veneto,2549,357,2906,88044,90950,-1226,3388,182506,7263,NaN,NaN,280719,3430733,1250287.0,31.0,NaN,NaN
4,2021-01-08T17:00:00,6,Friuli Venezia Giulia,671,64,735,11841,12576,211,633,40138,1862,NaN,NaN,54576,973526,351423.0,4.0,NaN,A seguito della periodica verifica dei casi se...
5,2021-01-08T17:00:00,7,Liguria,717,67,784,4421,5205,-96,400,54689,2980,NaN,NaN,62874,735330,330608.0,7.0,I 175908 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-08T17:00:00,8,Emilia-Romagna,2691,245,2936,53912,56848,-37,2026,121855,8210,NaN,NaN,186913,2669612,1260158.0,20.0,NaN,NaN
7,2021-01-08T17:00:00,9,Toscana,807,139,946,8289,9235,-265,452,110885,3830,NaN,NaN,123950,1945061,1083608.0,4.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-08T17:00:00,10,Umbria,280,43,323,3785,4108,47,215,25598,648,NaN,NaN,30354,522585,258421.0,1.0,NaN,NaN
9,2021-01-08T17:00:00,11,Marche,532,68,600,13412,14012,430,564,29752,1676,NaN,NaN,45440,565788,337261.0,2.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-09T17:00:00,1,Piemonte,2689,172,2861,12968,15829,-1716,1532,182165,8165,NaN,NaN,206159,1734982,1006938.0,14.0,NaN,NaN
1,2021-01-09T17:00:00,2,Valle d'Aosta,50,1,51,386,437,12,22,6661,388,NaN,NaN,7486,64752,39706.0,0.0,NaN,NaN
2,2021-01-09T17:00:00,3,Lombardia,3577,456,4033,51404,55437,760,2506,416326,25728,NaN,NaN,497491,5023007,2664865.0,15.0,NaN,NaN
3,2021-01-09T17:00:00,4,Trentino-Alto Adige,541,72,613,12748,13361,80,522,39407,1784,0.0,0.0,54552,844639,321057.0,3.0,0,NaN
4,2021-01-09T17:00:00,5,Veneto,2540,349,2889,86562,89451,-1499,3100,187023,7345,NaN,NaN,283819,3451512,1255475.0,32.0,NaN,NaN
5,2021-01-09T17:00:00,6,Friuli Venezia Giulia,690,62,752,12138,12890,314,1015,40825,1876,NaN,NaN,55591,981950,353683.0,3.0,NaN,NaN
6,2021-01-09T17:00:00,7,Liguria,702,65,767,4377,5144,-61,526,55257,2999,NaN,NaN,63400,739849,332011.0,7.0,I 180827 test antigenici rapidi sono riferiti ...,NaN
7,2021-01-09T17:00:00,8,Emilia-Romagna,2679,240,2919,55449,58368,1520,1790,122054,8279,NaN,NaN,188701,2685079,1264605.0,15.0,NaN,NaN
8,2021-01-09T17:00:00,9,Toscana,792,139,931,8174,9105,-130,529,111534,3840,NaN,NaN,124479,1956024,1087156.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2021-01-09T17:00:00,10,Umbria,279,47,326,3911,4237,129,295,25760,652,NaN,NaN,30649,526153,259676.0,6.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2021-01-08T17:00:00,1,Piemonte,2757,176,2933,14612,17545,-154,...,178949,8133,NaN,NaN,204627,1723609,1003245.0,5.0,NaN,NaN
1,1,2021-01-08T17:00:00,2,Valle d'Aosta,54,1,55,370,425,-9,...,6651,388,NaN,NaN,7464,64477,39592.0,0.0,NaN,NaN
2,2,2021-01-08T17:00:00,3,Lombardia,3436,466,3902,50775,54677,435,...,414643,25665,NaN,NaN,494985,4998160,2656804.0,25.0,NaN,NaN
3,19,2021-01-08T17:00:00,4,Trentino-Alto Adige,563,70,633,12648,13281,84,...,38982,1767,0.0,0.0,54030,838765,319874.0,2.0,0,NaN
4,3,2021-01-08T17:00:00,5,Veneto,2549,357,2906,88044,90950,-1226,...,182506,7263,NaN,NaN,280719,3430733,1250287.0,31.0,NaN,NaN
5,4,2021-01-08T17:00:00,6,Friuli Venezia Giulia,671,64,735,11841,12576,211,...,40138,1862,NaN,NaN,54576,973526,351423.0,4.0,NaN,A seguito della periodica verifica dei casi se...
6,5,2021-01-08T17:00:00,7,Liguria,717,67,784,4421,5205,-96,...,54689,2980,NaN,NaN,62874,735330,330608.0,7.0,I 175908 test antigenici rapidi sono riferiti ...,NaN
7,6,2021-01-08T17:00:00,8,Emilia-Romagna,2691,245,2936,53912,56848,-37,...,121855,8210,NaN,NaN,186913,2669612,1260158.0,20.0,NaN,NaN
8,7,2021-01-08T17:00:00,9,Toscana,807,139,946,8289,9235,-265,...,110885,3830,NaN,NaN,123950,1945061,1083608.0,4.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2021-01-08T17:00:00,10,Umbria,280,43,323,3785,4108,47,...,25598,648,NaN,NaN,30354,522585,258421.0,1.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2021-01-09T17:00:00,1,Piemonte,2689,172,2861,12968,15829,-1716,1532,...,8165,NaN,NaN,206159,1734982,1006938.0,14.0,NaN,NaN,4341375
1,2021-01-09T17:00:00,2,Valle d'Aosta,50,1,51,386,437,12,22,...,388,NaN,NaN,7486,64752,39706.0,0.0,NaN,NaN,125501
2,2021-01-09T17:00:00,3,Lombardia,3577,456,4033,51404,55437,760,2506,...,25728,NaN,NaN,497491,5023007,2664865.0,15.0,NaN,NaN,10103969
3,2021-01-09T17:00:00,4,Trentino-Alto Adige,541,72,613,12748,13361,80,522,...,1784,0.0,0.0,54552,844639,321057.0,3.0,0,NaN,1074819
4,2021-01-09T17:00:00,5,Veneto,2540,349,2889,86562,89451,-1499,3100,...,7345,NaN,NaN,283819,3451512,1255475.0,32.0,NaN,NaN,4907704
5,2021-01-09T17:00:00,6,Friuli Venezia Giulia,690,62,752,12138,12890,314,1015,...,1876,NaN,NaN,55591,981950,353683.0,3.0,NaN,NaN,1211357
6,2021-01-09T17:00:00,7,Liguria,702,65,767,4377,5144,-61,526,...,2999,NaN,NaN,63400,739849,332011.0,7.0,I 180827 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2021-01-09T17:00:00,8,Emilia-Romagna,2679,240,2919,55449,58368,1520,1790,...,8279,NaN,NaN,188701,2685079,1264605.0,15.0,NaN,NaN,4467118
8,2021-01-09T17:00:00,9,Toscana,792,139,931,8174,9105,-130,529,...,3840,NaN,NaN,124479,1956024,1087156.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2021-01-09T17:00:00,10,Umbria,279,47,326,3911,4237,129,295,...,652,NaN,NaN,30649,526153,259676.0,6.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2021-01-09T17:00:00,1,Piemonte,2689,172,2861,12968,15829,-1716,1532,...,NaN,NaN,206159,1734982,1006938.0,14.0,NaN,NaN,4341375,575
1,2021-01-09T17:00:00,2,Valle d'Aosta,50,1,51,386,437,12,22,...,NaN,NaN,7486,64752,39706.0,0.0,NaN,NaN,125501,20
2,2021-01-09T17:00:00,3,Lombardia,3577,456,4033,51404,55437,760,2506,...,NaN,NaN,497491,5023007,2664865.0,15.0,NaN,NaN,10103969,1036
3,2021-01-09T17:00:00,4,Trentino-Alto Adige,541,72,613,12748,13361,80,522,...,0.0,0.0,54552,844639,321057.0,3.0,0,NaN,1074819,106
4,2021-01-09T17:00:00,5,Veneto,2540,349,2889,86562,89451,-1499,3100,...,NaN,NaN,283819,3451512,1255475.0,32.0,NaN,NaN,4907704,1016
5,2021-01-09T17:00:00,6,Friuli Venezia Giulia,690,62,752,12138,12890,314,1015,...,NaN,NaN,55591,981950,353683.0,3.0,NaN,NaN,1211357,180
6,2021-01-09T17:00:00,7,Liguria,702,65,767,4377,5144,-61,526,...,NaN,NaN,63400,739849,332011.0,7.0,I 180827 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2021-01-09T17:00:00,8,Emilia-Romagna,2679,240,2919,55449,58368,1520,1790,...,NaN,NaN,188701,2685079,1264605.0,15.0,NaN,NaN,4467118,634
8,2021-01-09T17:00:00,9,Toscana,792,139,931,8174,9105,-130,529,...,NaN,NaN,124479,1956024,1087156.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2021-01-09T17:00:00,10,Umbria,279,47,326,3911,4237,129,295,...,NaN,NaN,30649,526153,259676.0,6.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-09T17:00:00,5,Veneto,2540,349,2889,86562,89451,-1499,3100,...,34.35,-1482,-288,20779,5188.0,82,4517,14.92,0.06317,5.78
1,2021-01-09T17:00:00,3,Lombardia,3577,456,4033,51404,55437,760,2506,...,44.02,629,543,24847,8061.0,63,1683,10.09,0.02480,4.92
2,2021-01-09T17:00:00,19,Sicilia,1256,205,1461,38937,40398,726,1839,...,34.86,711,-3,10427,6356.0,31,1082,17.64,0.03701,2.14
3,2021-01-09T17:00:00,8,Emilia-Romagna,2679,240,2919,55449,58368,1520,1790,...,37.85,1537,-236,15467,4447.0,69,199,11.57,0.04007,4.22
4,2021-01-09T17:00:00,12,Lazio,2824,310,3134,75035,78169,314,1543,...,36.60,360,-70,11982,5688.0,54,1175,12.88,0.02631,3.03
5,2021-01-09T17:00:00,1,Piemonte,2689,172,2861,12968,15829,-1716,1532,...,29.91,-1644,687,11373,3693.0,32,3216,13.47,0.03529,4.75
6,2021-01-09T17:00:00,16,Puglia,1376,175,1551,53184,54735,813,1499,...,47.43,787,150,10880,4317.0,27,659,13.78,0.03740,2.51
7,2021-01-09T17:00:00,15,Campania,1376,107,1483,73793,75276,130,1263,...,21.19,142,279,14714,6482.0,36,1097,8.58,0.02183,3.43
8,2021-01-09T17:00:00,6,Friuli Venezia Giulia,690,62,752,12138,12890,314,1015,...,34.44,297,382,8424,2260.0,14,687,12.05,0.08379,4.59
9,2021-01-09T17:00:00,11,Marche,529,71,600,14006,14606,594,688,...,49.65,594,124,5311,4024.0,9,85,12.95,0.04531,3.04


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,72,34,106,67.92
1,Marche,71,72,143,49.65
2,Puglia,175,194,369,47.43
3,Lombardia,456,580,1036,44.02
4,Umbria,47,70,117,40.17
5,Emilia-Romagna,240,394,634,37.85
6,Lazio,310,537,847,36.60
7,Sicilia,205,383,588,34.86
8,Friuli Venezia Giulia,62,118,180,34.44
9,Veneto,349,667,1016,34.35


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,2021-01-07T17:00:00,22,P.A. Trento,361,47,408,1538,1946,-78,170,20133,994,NaN,NaN,23073,463967,153931.0,3.0,NaN,"Dei 170 casi odierni, 98 sono molecolari con p..."
638,2021-01-08T17:00:00,21,P.A. Bolzano,208,24,232,11119,11351,100,163,18757,769,NaN,NaN,30877,373746,165782.0,1.0,NaN,NaN
639,2021-01-08T17:00:00,22,P.A. Trento,355,46,401,1529,1930,-16,80,20225,998,NaN,NaN,23153,465019,154092.0,1.0,NaN,"Dei 80 casi odierni, 28 sono molecolari con pr..."
640,2021-01-09T17:00:00,21,P.A. Bolzano,208,24,232,11291,11523,172,380,18956,778,NaN,NaN,31257,376673,166561.0,0.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,2021-01-05T17:00:00,4,Trentino-Alto Adige,574,72,646,12343,12989,100,376,...,10.0,3200.0,664.0,98.0,11.15,4.97,11.75,56.63,5.88,0.78
317,2021-01-06T17:00:00,4,Trentino-Alto Adige,562,72,634,12495,13129,140,523,...,9.0,5449.0,984.0,152.0,11.36,4.83,9.60,53.15,0.00,1.08
318,2021-01-07T17:00:00,4,Trentino-Alto Adige,566,72,638,12559,13197,68,433,...,6.0,4361.0,960.0,64.0,11.29,4.83,9.93,45.10,0.00,0.52
319,2021-01-08T17:00:00,4,Trentino-Alto Adige,563,70,633,12648,13281,84,243,...,10.0,2716.0,576.0,89.0,11.06,4.77,8.95,42.19,-2.78,0.64


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
